In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
np.random.seed(42) 


In [ ]:
#load and read necessary files
train=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv')
test=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv')
movies=pd.read_csv('/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv')
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [ ]:
train.shape

In [ ]:
import regex as re 
stop_words = ['this', 'is', 'a', 'the', 'it', 'at', 'all', 'was'
        
              # Add more stop words as needed 
             ]










In [ ]:
test.shape

In [ ]:
train=train.dropna()
X = train.drop('sentiment', axis="columns")
Y = train['sentiment']

In [ ]:
X

In [ ]:
def remove_stop_words(text, stop_words):
    pattern = r'\b(?:' + '|'.join(stop_words) + r')\b'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    return cleaned_text

In [ ]:
X

In [ ]:
test["reviewText"].fillna("", inplace=True)

Dropping duplicates from movies to merge it with train

In [ ]:
movies.drop_duplicates(subset=['movieid'], keep='first', inplace=True)

In [ ]:
merged= train.merge(movies[['movieid', 'audienceScore']], on='movieid', how='left')

In [ ]:
merged

INSIGHTS AFTER CONDUCTING EDA:
features like 'movieid', 'reviewerName' and 'isFrequentReviewer' have weak correlation with the Label vector

In [ ]:
merged.drop(['movieid', 'reviewerName', 'isFrequentReviewer'], axis=1, inplace=True)

In [ ]:
X_numerical = pd.DataFrame(merged["audienceScore"])

In [ ]:
merged

In [ ]:
X_numerical

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(5, 3))
sns.histplot(data=X_numerical, x='audienceScore', bins=20, kde=True)
plt.xlabel('Audience Score')
plt.ylabel('Frequency')
plt.title('Distribution of Audience Scores')
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
sns.boxplot(data=X_numerical, y='audienceScore')
plt.ylabel('Audience Score')
plt.title('Box Plot of Audience Scores')
plt.show()

INSIGHTS:
outliers present in the minimum region only 

In [ ]:
X_numerical['audienceScore'].values.reshape(-1, 1)

Applying Simple Imputer

In [ ]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values= np.nan , strategy='mean')
X_numerical= pd.DataFrame(SimpleImputer().fit_transform(X_numerical))

In [ ]:
X_numerical

Vectorizing Data using TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
vectorizer.fit(X["reviewText"])
X_tfidf=vectorizer.transform(merged["reviewText"])

Using Robust Scaling to prevent overfitting of model

In [ ]:
'''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
scaled_tfidf_matrix = scaler.fit_transform(X_tfidf)'''

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(with_centering=False)
scaled_tfidf_matrix = scaler.fit_transform(X_tfidf)

In [ ]:
mergeTest= test.merge(movies[['movieid', 'audienceScore']], on='movieid', how='left')

Combining both preprocessed parts

In [ ]:
import scipy.sparse as sp
X_combined = sp.hstack((X_numerical, scaled_tfidf_matrix), format='csr')

In [ ]:
merged.isnull().sum()

In [ ]:
Test_Tfidf=vectorizer.transform(test["reviewText"])

Train Test Split for evaluation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)

Model 1: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(penalty='l2', tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=1000, multi_class='ovr', verbose=0, warm_start=True, n_jobs=-1, l1_ratio=None)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
#print(report)

  precision    recall  f1-score   support

    NEGATIVE       0.77      0.62      0.69     10148
    POSITIVE       0.83      0.91      0.87     21115

    accuracy                           0.82     31263
   macro avg       0.80      0.77      0.78     31263
weighted avg       0.81      0.82      0.81     31263

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss='log_loss', random_state=42)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
report = classification_report(y_test,y_pred)
#print(report)

              precision    recall  f1-score   support

    NEGATIVE       0.62      0.43      0.51     10148
    POSITIVE       0.76      0.87      0.81     21115

    accuracy                           0.73     31263
   macro avg       0.69      0.65      0.66     31263
weighted avg       0.71      0.73      0.71     31263

In [ ]:
from sklearn.neural_network import MLPClassifier 
model = MLPClassifier(hidden_layer_sizes=(10, ), activation='logistic', solver='lbfgs',max_iter=2000,warm_start=False)

below commented code is the original parameters used for HPT

In [ ]:

'''from sklearn.model_selection import GridSearchCV
param_grid= [
    {'penalty': ['l1', 'l2', 'elasticnet'],
     'C': np.logspace(-4, 4, 9),
    'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 500, 1000, 2000]}
]
clf=GridSearchCV(model, param_grid=param_grid, cv=3, verbose=True, n_jobs=-1)'''


In [ ]:
'''best_clf=clf.fit(X, Y)
best_clf.best_estimator_'''

In [ ]:
'''param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}'''

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
report = classification_report(y_test,y_pred)
print(report)

 precision    recall  f1-score   support

    NEGATIVE       0.74      0.67      0.70     10148
    POSITIVE       0.85      0.89      0.87     21115

    accuracy                           0.82     31263
   macro avg       0.79      0.78      0.79     31263
weighted avg       0.81      0.82      0.81     31263

INSIGHTS AFTER TRAINING THE 3 MODELS:
eventhough logistic regression , mlpclassifier had similar evaluation scores after HPT

MLPclassifer performed better on test data indicating logistic regression was overfitting

Hence, MLPclassifier will be used as our final model

Also Robust Scaling helped in optimizing MLPclassifier as MLPclassifier is sensitive to outliers

Preprocessing Test Data (Repeating steps used for pre processing training data)

In [ ]:
mergeTest.drop(['reviewerName', 'isTopCritic'], axis=1, inplace=True)

In [ ]:
Test_numerical=pd.DataFrame((mergeTest["audienceScore"]))

In [ ]:
Test_numerical['audienceScore'].values.reshape(-1, 1)

In [ ]:
Test_numerical.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values= np.nan , strategy='mean')
Test_numerical= pd.DataFrame(SimpleImputer().fit_transform(Test_numerical))

In [ ]:
Test_Tfidf_matrix = scaler.fit_transform(Test_Tfidf)

In [ ]:
Test_numerical.isnull().sum()

In [ ]:
T_combined = sp.hstack((Test_numerical, Test_Tfidf_matrix), format='csr')

In [ ]:
T_combined.shape

In [ ]:
f_pred=model.predict(T_combined)

In [ ]:
submission=pd.DataFrame(columns=['id', 'sentiment'])
submission['id']=[i for i in range(len(f_pred))]
submission["sentiment"]=[i for i in f_pred]
submission.to_csv('submission.csv',index=False)
submission